# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
city_data=pd.read_csv("City_Data_Output.csv")
city_data=city_data[["City","Id","country","lat","lng","Cloudiness","humidity","temperature","wind_speed"]]

len(city_data["City"])

554

In [4]:
fig=gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
city_data.head()


,City,Id,country,lat,lng,Cloudiness,humidity,temperature,wind_speed
21,Rosignol,3376223,GY,6.2833,-57.5333,0,85,78.10,6.06
201,Jalu,86049,LY,29.0331,21.5482,0,55,74.59,7.56
252,Ikalamavony,1064258,MG,-21.1500,46.5833,0,43,75.74,4.85
298,Caravelas,3466980,BR,-17.7125,-39.2481,0,85,70.43,5.97
358,Saint-Pierre,935214,RE,-21.3393,55.4781,0,69,76.91,5.75


In [23]:
city_data["humidity"].max()

100

In [6]:
location=city_data[["lat","lng"]].astype(float)
weight_humidity=city_data["humidity"].astype(float)


In [25]:
location=city_data[["lat","lng"]].astype(float)
city_humidity=city_data["humidity"].astype(float)

heat_layer=gmaps.heatmap_layer(location,weights=weight_humidity,dissipating=False,
                               max_intensity=100,point_radius=2)
fig.add_layer(heat_layer)
fig
                              

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
filt=((city_data["temperature"]>70)&
     (city_data["temperature"]<80)&
     (city_data["wind_speed"]<10)&
     (city_data["Cloudiness"]==0))

city_data=city_data.loc[filt,:]
city_data.dropna()
        
city_data

,City,Id,country,lat,lng,Cloudiness,humidity,temperature,wind_speed
21,Rosignol,3376223,GY,6.2833,-57.5333,0,85,78.10,6.06
201,Jalu,86049,LY,29.0331,21.5482,0,55,74.59,7.56
252,Ikalamavony,1064258,MG,-21.1500,46.5833,0,43,75.74,4.85
298,Caravelas,3466980,BR,-17.7125,-39.2481,0,85,70.43,5.97
358,Saint-Pierre,935214,RE,-21.3393,55.4781,0,69,76.91,5.75
379,Chifeng,2038067,CN,42.2683,118.9636,0,33,72.77,1.03
418,Tūkrah,88834,LY,32.5341,20.5791,0,66,77.29,7.83
428,Sirte,2210554,LY,31.2089,16.5887,0,65,75.00,8.12
440,Ziarat,1162094,PK,30.3823,67.7282,0,11,75.70,5.01
461,Talgar,1518518,KZ,43.3027,77.2395,0,13,78.96,4.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df=city_data[["City","country","lat","lng"]]
hotel_df=hotel_df.reset_index(drop=True)
hotel_df["hotel name"]=""

hotel_df

,City,country,lat,lng,hotel name
0,Rosignol,GY,6.2833,-57.5333,
1,Jalu,LY,29.0331,21.5482,
2,Ikalamavony,MG,-21.1500,46.5833,
3,Caravelas,BR,-17.7125,-39.2481,
4,Saint-Pierre,RE,-21.3393,55.4781,
5,Chifeng,CN,42.2683,118.9636,
6,Tūkrah,LY,32.5341,20.5791,
7,Sirte,LY,31.2089,16.5887,
8,Ziarat,PK,30.3823,67.7282,
9,Talgar,KZ,43.3027,77.2395,


In [16]:
for index,row in hotel_df.iterrows():
    try:
        lat=row["lat"]
        lng=row["lng"]
        location=f"{lat},{lng}"
        base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        parameters={"location":location,"radius":5000,"keyword":"Hotel","key":g_key}
        repo=requests.get(base_url,params=parameters).json()
    
        hotel_df.loc[index,"hotel name"]=repo['results'][0]["name"]
        
    except IndexError:
        
        print("data not found")
            

data not found


In [17]:
hotel_df

,City,country,lat,lng,hotel name
0,Rosignol,GY,6.2833,-57.5333,"Leisure Inn, Hotel | Restaurant"
1,Jalu,LY,29.0331,21.5482,Jalu Hotel
2,Ikalamavony,MG,-21.1500,46.5833,
3,Caravelas,BR,-17.7125,-39.2481,Pousada Liberdade
4,Saint-Pierre,RE,-21.3393,55.4781,Le Battant Des Lames
5,Chifeng,CN,42.2683,118.9636,Wanda Realm Chifeng
6,Tūkrah,LY,32.5341,20.5791,مصيف مرحبا للعائلات
7,Sirte,LY,31.2089,16.5887,Al- Waffaa Hotel
8,Ziarat,PK,30.3823,67.7282,Grand Hotel
9,Talgar,KZ,43.3027,77.2395,Alma Port


In [18]:
hotel_df["hotel name"].replace("",np.nan,inplace=True)

hotel_df.dropna(inplace=True)
hotel_df.rename(columns={"country":"Country",
                        "hotel name":"Hotel Name"},inplace=True)
hotel_df


,City,Country,lat,lng,Hotel Name
0,Rosignol,GY,6.2833,-57.5333,"Leisure Inn, Hotel | Restaurant"
1,Jalu,LY,29.0331,21.5482,Jalu Hotel
3,Caravelas,BR,-17.7125,-39.2481,Pousada Liberdade
4,Saint-Pierre,RE,-21.3393,55.4781,Le Battant Des Lames
5,Chifeng,CN,42.2683,118.9636,Wanda Realm Chifeng
6,Tūkrah,LY,32.5341,20.5791,مصيف مرحبا للعائلات
7,Sirte,LY,31.2089,16.5887,Al- Waffaa Hotel
8,Ziarat,PK,30.3823,67.7282,Grand Hotel
9,Talgar,KZ,43.3027,77.2395,Alma Port
10,Iquitos,PE,-3.7481,-73.2472,Nativo Hotel


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [26]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))